In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, get_object, save, Logger, DigitalLog
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [3]:
player = 'Axis'

In [4]:
out = format_output(init_game(debug=False, player=player))

Beginning phase: Setup
Traceback (most recent call last):
  File "/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/Other/tnt/passive_backend.py", line 143, in step
    possible_actions = phase(G, player, WAITING[player].actions, action) # already factorized
  File "/mnt/c/Users/anwan/OneDrive - UW Office 365/Khan/Other/tnt/tnt_setup.py", line 254, in setup_phase
    assert action in options, 'Invalid action: {}'.format(action)
AssertionError: Invalid action: ('Britain', 'Paris', 'Infantry')

('France', 'Paris', 'Infantry')
('Britain', 'Bombay', 'AirForce')
('Britain', 'Delhi', 'Infantry')
('France', 'Paris', 'Infantry')
('Britain', 'Suez', 'Submarine')
('France', 'Algiers', 'Submarine')
('Britain', 'London', 'Tank')
('France', 'Marseille', 'Carrier')
('Britain', 'Delhi', 'Tank')
('Britain', 'London', 'Carrier')
('Britain', 'London', 'Submarine')
('Britain', 'Glasgow', 'Submarine')
Player West is waiting


In [5]:
player = 'West'

In [12]:
out = format_output(get_status(player))

In [7]:
render_dict(out)

In [8]:
# action = random.choice(list(util.decode_actions(out.actions)))
# action

In [9]:
#out = take_action(player, action)

In [10]:
out = format_output(take_action(player, ('Britain', 'Paris', 'Infantry')))

In [14]:
for _ in range(10):
    if 'error' in out:
        print(out.error)
        break
    elif 'actions' not in out:
        #render_dict(out)
        print('Player {} is waiting'.format(player))
        break
    action = random.choice(list(util.decode_actions(out.actions)))
    print(action)
    out = format_output(take_action(player, action))

In [15]:
G = get_G()
render_dict(G)

In [33]:
W = get_waiting_actions()
render_dict(W)